In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🤖 Robots vs. Labor: A CES Production Model

How does the rise of automation and "robots" (representing capital that can potentially substitute for human labor) affect production? The **Constant Elasticity of Substitution (CES)** production function provides a flexible framework to analyze this.

Unlike the Cobb-Douglas function (which assumes a constant elasticity of substitution equal to 1), the CES function allows this elasticity to vary, enabling us to model scenarios where robots and labor are either close substitutes or poor substitutes.

This simulation explores how output evolves when the stocks of robots ($R$) and labor ($L$) grow at potentially different rates, governed by a CES production function.

# ⚙️ The CES Production Function

The CES production function takes the form:

$$Y = A \left[ \beta R^{\rho} + (1 - \beta) L^{\rho} \right]^{1/\rho}$$

Where:
- $Y$: Total output
- $A$: Total Factor Productivity (efficiency parameter)
- $R$: Input 1 (e.g., Robot capital stock)
- $L$: Input 2 (e.g., Labor)
- $\beta$: Distribution parameter, indicating the relative importance or share of input R ($0 < \beta < 1$). $(1-\beta)$ is the share parameter for L.
- $\rho$: Substitution parameter ($\rho \le 1$, and $\rho \neq 0$). This parameter determines how easily inputs can be substituted for one another.

**Elasticity of Substitution ($\sigma$):**
The key parameter derived from $\rho$ is the elasticity of substitution:
$$ \sigma = \frac{1}{1 - \rho} $$
$\sigma$ measures the percentage change in the input ratio ($R/L$) for a one percent change in the ratio of their marginal products (the Marginal Rate of Technical Substitution, MRTS), holding output constant.
- **If $\rho \to 1$ ($\sigma \to \infty$):** Inputs are perfect substitutes.
- **If $\rho \to 0$ ($\sigma \to 1$):** The CES function approaches the Cobb-Douglas form. Inputs are imperfect substitutes with unit elasticity.
- **If $\rho \to -\infty$ ($\sigma \to 0$):** Inputs are perfect complements (Leontief function, used in fixed proportions).

In this simulation:
* We typically focus on $\rho < 1$.
* A value of $\rho$ closer to 1 means $\sigma$ is large, indicating high substitutability (robots easily replace labor).
* A value of $\rho$ that is large and negative means $\sigma$ is small (close to 0), indicating low substitutability (robots and labor are more like complements).

The simulation tracks the growth of $R$ and $L$ at specified rates and calculates the resulting output $Y$ using the CES function.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def ces_robots_simulation(beta=0.5, A=1.0, R0=1.0, L0=100.0,
                          gR=0.04, gL=0.01, rho=-1.0, T=30):
    """
    Simulates output growth using a CES production function with
    two inputs (Robots R, Labor L) growing at constant rates.
    Y = A * [beta * R^rho + (1-beta) * L^rho]^(1/rho)

    Args:
        beta (float): Distribution parameter for Robots (0 < beta < 1).
        A (float): Total Factor Productivity.
        R0 (float): Initial stock of Robots.
        L0 (float): Initial stock of Labor.
        gR (float): Growth rate of Robot stock.
        gL (float): Growth rate of Labor stock.
        rho (float): Substitution parameter (rho <= 1, rho != 0).
        T (int): Time Horizon (number of years).
    """
    # Input validation
    T = int(T)
    if T < 1: T = 1
    beta = np.clip(beta, 0.01, 0.99)
    A = max(A, 0.1)
    R0 = max(R0, 0.1)
    L0 = max(L0, 0.1)
    # Rho cannot be exactly zero for the standard CES formula
    if abs(rho) < 1e-6:
         # If rho is very close to zero, use Cobb-Douglas as the limit
         rho = 1e-6 # Use a tiny value instead of zero, or implement Cobb-Douglas case
         warnings.warn("Rho is close to zero; CES approaches Cobb-Douglas.")
    elif rho > 1:
         rho = 0.99 # Rho > 1 is unusual, limit it
         warnings.warn("Rho > 1 is unusual for standard CES; limited to 0.99.")


    # --- Calculate Elasticity of Substitution ---
    sigma = 1.0 / (1.0 - rho) if (1.0 - rho) != 0 else np.inf

    # --- Simulate Levels ---
    time = np.arange(T + 1) # T+1 points for levels
    R = R0 * (1 + gR)**time
    L = L0 * (1 + gL)**time

    # Calculate Output Y using CES function
    # Handle potential overflow/underflow with large rho or large R, L
    try:
        # Use log-sum-exp trick for numerical stability if rho is large negative?
        # Or calculate directly with checks
        term_R = beta * R**rho
        term_L = (1 - beta) * L**rho
        # Check for non-positive terms if rho is negative and R/L are small
        if rho < 0:
            term_R[R < 1e-9] = np.inf # Avoid issues with R^rho if R is tiny
            term_L[L < 1e-9] = np.inf
        if rho == 1e-6: # Cobb-Douglas Approximation for rho near 0
             Y = A * (R**beta) * (L**(1-beta))
        else:
             inside = term_R + term_L
             # Check if 'inside' is positive before raising to power 1/rho
             if np.any(inside <= 0):
                  Y = np.zeros_like(time, dtype=float) # Or handle error
                  warnings.warn("Non-positive term inside CES bracket; output may be incorrect.")
             else:
                  Y = A * inside**(1.0 / rho)

    except (OverflowError, ValueError):
        Y = np.full(T + 1, np.nan) # Indicate calculation failure
        warnings.warn("Numerical error during CES calculation. Check parameters.")

    # Calculate Output per Worker
    Y_per_L = Y / L if L0 > 0 else np.full(T + 1, np.nan)


    # --- Plotting ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()

    # Plot 0: Inputs (Log Scale)
    axes[0].plot(time, R, label=f'Robots R (gR={gR:.1%})', color='firebrick', lw=2)
    axes[0].plot(time, L, label=f'Labor L (gL={gL:.1%})', color='navy', lw=2, linestyle='--')
    axes[0].set_yscale('log')
    axes[0].set_title("Input Growth (Log Scale)")
    axes[0].set_xlabel("Time (Years)")
    axes[0].set_ylabel("Input Quantity (Log Scale)")
    axes[0].legend()
    axes[0].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 1: Output (Log Scale)
    axes[1].plot(time, Y, label='Total Output (Y)', color='darkgreen', lw=2)
    axes[1].set_yscale('log')
    axes[1].set_title("Output Growth (Log Scale)")
    axes[1].set_xlabel("Time (Years)")
    axes[1].set_ylabel("Output Y(t) (Log Scale)")
    axes[1].legend()
    axes[1].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 2: Output per Worker (Log Scale)
    axes[2].plot(time, Y_per_L, label='Output per Worker (Y/L)', color='purple', lw=2)
    axes[2].set_yscale('log')
    axes[2].set_title("Output per Worker (Log Scale)")
    axes[2].set_xlabel("Time (Years)")
    axes[2].set_ylabel("Y/L (Log Scale)")
    axes[2].legend()
    axes[2].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 3: Ratio of Inputs (R/L)
    input_ratio = R / L
    axes[3].plot(time, input_ratio, label='Robot/Labor Ratio (R/L)', color='teal', lw=2)
    axes[3].set_yscale('log') # Often useful to see ratio growth on log scale
    axes[3].set_title("Input Ratio (Log Scale)")
    axes[3].set_xlabel("Time (Years)")
    axes[3].set_ylabel("R / L (Log Scale)")
    axes[3].legend()
    axes[3].grid(True, which='both', linestyle='--', alpha=0.6)


    fig.suptitle(f"CES Production: Robots vs Labor (σ = {sigma:.2f})", fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Display Parameters ---
    summary_md = f"""
    ### ⚙️ Model Parameters & Results:

    * **CES Function:** $Y = A [\\beta R^{{\\rho}} + (1 - \\beta) L^{{\\rho}}]^{{1/\\rho}}$
    * **Parameters:** A={A:.2f}, β={beta:.2f}, ρ={rho:.2f}
    * **Elasticity of Substitution (σ = 1/(1-ρ)):** {sigma:.3f}
        * *(σ=1: Cobb-Douglas, σ>1: High substitutability, σ<1: Low substitutability)*
    * **Input Growth Rates:** gR={gR:.2%}, gL={gL:.2%}
    * **Final Output (Y({T})):** {Y[-1]:,.2f}
    * **Final Output per Worker (Y({T})/L({T})):** {Y_per_L[-1]:,.2f}
    """
    display(Markdown(summary_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    ces_robots_simulation,
    beta=FloatSlider(value=0.5, min=0.05, max=0.95, step=0.05, description='Robot Share (beta β):', style=style, layout=layout),
    A=FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='TFP (A):', style=style, layout=layout),
    R0=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Initial Robots (R0):', style=style, layout=layout),
    L0=FloatSlider(value=100.0, min=10, max=1000, step=10, description='Initial Labor (L0):', style=style, layout=layout, readout_format=',.0f'),
    gR=FloatSlider(value=0.04, min=0.0, max=0.1, step=0.005, description='Robot Growth (gR):', style=style, layout=layout, readout_format='.1%'),
    gL=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.005, description='Labor Growth (gL):', style=style, layout=layout, readout_format='.1%'),
    rho=FloatSlider(value=-1.0, min=-5.0, max=0.9, step=0.1, description='Subst. Param (rho ρ):', style=style, layout=layout),
    T=IntSlider(value=30, min=10, max=100, step=1, description='Years (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=0.5, description='Robot Share (beta β):', layout=Layout(width='95%'), …

# 🤖 Model 8: Robots and the Future of Work (CES Production)

This model explores the impact of robots on output over time using a CES production function:

\[
Y_t = A \left[ \beta R_t^{\rho} + (1 - \beta) L_t^{\rho} \right]^{1/\rho}
\]

- **R** = Robot capital
- **L** = Labor input
- **β** = Robot's share in production
- **ρ** = Degree of substitutability (ρ < 1 = imperfect)

# Insights:
- High robot growth and substitutability can lead to rising output even with stagnant labor
- Labor becomes less important when β is high and ρ approaches 0
- Use this to simulate fears of automation replacing work — or augmenting it